In [1]:
import pandas as pd

In [2]:
pc = pd.read_excel('pelecard_data.xlsx', dtype='str')

In [3]:
#pc,

In [4]:
pc.columns

Index(['מספר מסוף', 'שם עסק', 'תאריך ושעה', 'מספר ריכוז', 'מספר שובר',
       'מספר כרטיס אשראי', 'תוקף', 'חברה סולקת', 'מותג', 'סוג עסקה',
       'מספר אישור', 'סוג אשראי', 'סכום', 'מטבע', 'מספר תשלומים',
       'תשלום ראשון', 'תשלום קבוע', 'פרטים נוספים', 'חברה מנפיקה', 'ת.ז',
       'מזהה עסקה', 'טוקן', 'מקור כרטיס האשראי', 'סטאטוס', 'קוד סטאטוס'],
      dtype='object')

In [5]:
col_names_to_english = {
                        'מספר מסוף': 'aaa', 
                        'שם עסק': 'bbb', 
                        'תאריך ושעה': 'trx_time', 
                        'מספר ריכוז': 'conc_num', 
                        'מספר שובר': 'voucher_num', 
                        'מספר כרטיס אשראי': 'card_num', 
                        'תוקף': 'card_exp', 
                        'חברה סולקת': 'ccc', 
                        'מותג': 'brand', 
                        'סוג עסקה': 'pay_way', 
                        'מספר אישור': 'appr_num', 
                        'סוג אשראי': 'trx_type', 
                        'סכום': 'trx_amount', 
                        'מטבע': 'currency', 
                        'מספר תשלומים': 'split_pay', 
                        'תשלום ראשון': 'ddd', 
                        'תשלום קבוע': 'eee', 
                        'פרטים נוספים': 'note', 
                        'חברה מנפיקה': 'fff', 
                        'ת.ז': 'guest_id', 
                        'מזהה עסקה': 'uid', 
                        'טוקן': 'token', 
                        'מקור כרטיס האשראי': 'iin', 
                        'סטאטוס': 'status', 
                        'קוד סטאטוס': 'status_code'
    }

In [6]:
pc.rename(columns=col_names_to_english, inplace=True)

In [7]:
cols_to_drop = ['aaa', 'bbb', 'voucher_num', 'card_exp', 'ccc', 'pay_way', 'trx_type', 
                'ddd', 'eee', 'fff', 'guest_id', 'uid', 'iin', 'status', 'status_code', 
                'token', 'conc_num']

In [8]:
pc.drop(cols_to_drop, axis='columns', inplace=True)

In [9]:
needed_columns = ['trx_amount', 'card_num', 'trx_time', 'note', 'currency', 'appr_num', 'split_pay', 'brand']

In [10]:
pc = pc[needed_columns]

In [11]:
pc.isna().any()

trx_amount    False
card_num      False
trx_time      False
note           True
currency      False
appr_num      False
split_pay     False
brand         False
dtype: bool

In [12]:
pc.fillna('0', inplace=True)

In [13]:
# replace hebrew credit card company names with english ones in column 'brand'
pc['brand'] = pc['brand'].replace(r'מסטרכרד|מאסטרו', 'mastercard', regex=True)
pc['brand'] = pc['brand'].replace('ויזה', 'visa')
pc['brand'] = pc['brand'].replace('אמקס', 'visa')
pc['brand'] = pc['brand'].replace('דיינרס', 'diners')

In [14]:
# make notes lowercase for convenience
pc['note'] = pc['note'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [15]:
# some improvements in column values
pc['trx_time'] = pc['trx_time'].replace(r'^[0-9/ ]+([0-9]{2}:[0-9]{2}):[0-9]{2}', r'\1', regex=True)

In [16]:
pc['card_num'] = pc['card_num'].replace(r'^[0-9]+[*]+([0-9]{4})', r'xx\1', regex=True)

In [17]:
pc['currency'] = pc['currency'].replace('$', 'usd')

In [18]:
pc['currency'] = pc['currency'].replace('₪', 'ils')

In [19]:
pc['note'] = pc['note'].replace(r'([0-9]{7,}) [0-9]+', r'\1', regex=True)

In [20]:
pc['note'] = pc['note'].replace(r'^b[0-9]{5,}([0-9]{3}$)', r'pos xx\1', regex=True)

In [21]:
# adding "кракозябра" just because it sounds funny
pc['note'] = pc['note'].replace(r'^�$', 'pos кракозябра', regex=True)

In [22]:
# trim whitespaces
for column in pc.columns:
    pc[column] = pc[column].apply(lambda x: x.strip())

In [23]:
rate = 3.75

In [24]:
pc['split_pay'] = pc['split_pay'].astype('int8')

In [25]:
pc['trx_amount'] = pc['trx_amount'].astype('float')

In [26]:
pc['ils'] = pc.apply(lambda x: round(x['trx_amount'], 2) 
                     if x['currency'] == 'ils' 
                     else round(x['trx_amount'] * rate, 2), axis='columns')

In [27]:
pc['usd'] = pc.apply(lambda x: round(x['trx_amount'], 2) 
                     if x['currency'] == 'usd' 
                     else round(x['trx_amount'] / rate, 2), axis='columns')

In [28]:
pc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   trx_amount  53 non-null     float64
 1   card_num    53 non-null     object 
 2   trx_time    53 non-null     object 
 3   note        53 non-null     object 
 4   currency    53 non-null     object 
 5   appr_num    53 non-null     object 
 6   split_pay   53 non-null     int8   
 7   brand       53 non-null     object 
 8   ils         53 non-null     float64
 9   usd         53 non-null     float64
dtypes: float64(3), int8(1), object(6)
memory usage: 3.9+ KB


In [29]:
pc.drop('trx_amount', axis='columns', inplace=True)

In [30]:
chosen_columns = ['usd', 'ils', 'card_num', 'trx_time', 'note', 'currency', 'appr_num', 'split_pay', 'brand']

In [31]:
pc = pc[chosen_columns]

In [32]:
pc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   usd        53 non-null     float64
 1   ils        53 non-null     float64
 2   card_num   53 non-null     object 
 3   trx_time   53 non-null     object 
 4   note       53 non-null     object 
 5   currency   53 non-null     object 
 6   appr_num   53 non-null     object 
 7   split_pay  53 non-null     int8   
 8   brand      53 non-null     object 
dtypes: float64(2), int8(1), object(6)
memory usage: 3.5+ KB
